# 《心境流转》系统初始化

本notebook用于初始化和验证"心境流转"睡前音画疗愈系统的运行环境。

## 🎯 目标
- 检测JupyterHub环境和硬件资源
- 验证所需依赖库的安装
- 配置系统参数和路径
- 进行基础功能测试

## ⚠️ 重要提示
- 首次运行此系统时必须执行此notebook
- 确保具有足够的GPU内存 (40GB+)
- 某些模型可能需要较长时间下载

## 1. 环境检测

In [ ]:
import sys
import os
import platform
import subprocess
from pathlib import Path
import json
from datetime import datetime

print("=" * 60)
print("《心境流转》系统环境检测")
print("=" * 60)

# 基础环境信息
print(f"Python 版本: {sys.version}")
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"CPU架构: {platform.machine()}")
print(f"当前工作目录: {os.getcwd()}")
print(f"Python路径: {sys.executable}")
print(f"检测时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# 设置项目根目录
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print(f"项目根目录: {PROJECT_ROOT}")

# 添加项目路径到sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    print("✅ 项目路径已添加到sys.path")

# 检查关键目录
key_directories = [
    'src',
    'configs', 
    'research/theory',
    'notebooks',
    'outputs'
]

print("\n📁 关键目录检查:")
for directory in key_directories:
    dir_path = PROJECT_ROOT / directory
    if dir_path.exists():
        print(f"✅ {directory}")
    else:
        print(f"❌ {directory} (缺失)")
        # 创建缺失的输出目录
        if directory == 'outputs':
            dir_path.mkdir(parents=True, exist_ok=True)
            print(f"  📁 已创建输出目录: {dir_path}")

## 2. 硬件资源检测

In [ ]:
import psutil
import gc

print("\n🖥️ 系统资源检测:")
print("-" * 40)

# CPU信息
cpu_count = psutil.cpu_count()
cpu_freq = psutil.cpu_freq()
print(f"CPU核心数: {cpu_count}")
if cpu_freq:
    print(f"CPU频率: {cpu_freq.current:.2f} MHz")

# 内存信息
memory = psutil.virtual_memory()
memory_gb = memory.total / (1024**3)
memory_available_gb = memory.available / (1024**3)
print(f"总内存: {memory_gb:.1f} GB")
print(f"可用内存: {memory_available_gb:.1f} GB")
print(f"内存使用率: {memory.percent}%")

# 磁盘信息
disk = psutil.disk_usage('/')
disk_total_gb = disk.total / (1024**3)
disk_free_gb = disk.free / (1024**3)
print(f"磁盘总容量: {disk_total_gb:.1f} GB")
print(f"磁盘可用: {disk_free_gb:.1f} GB")

# 资源充足性评估
print("\n📊 资源评估:")
if memory_gb >= 32:
    print("✅ 内存充足 (>=32GB)")
elif memory_gb >= 16:
    print("⚠️ 内存可用但可能不足 (16-32GB)")
else:
    print("❌ 内存不足 (<16GB)")

if disk_free_gb >= 50:
    print("✅ 磁盘空间充足 (>=50GB)")
elif disk_free_gb >= 20:
    print("⚠️ 磁盘空间紧张 (20-50GB)")
else:
    print("❌ 磁盘空间不足 (<20GB)")

In [ ]:
# GPU检测
print("\n🎮 GPU检测:")
print("-" * 40)

try:
    import torch
    
    print(f"PyTorch版本: {torch.__version__}")
    
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        print(f"✅ CUDA可用，发现 {gpu_count} 个GPU")
        print(f"CUDA版本: {torch.version.cuda}")
        
        for i in range(gpu_count):
            props = torch.cuda.get_device_properties(i)
            total_memory = props.total_memory / (1024**3)
            allocated_memory = torch.cuda.memory_allocated(i) / (1024**3)
            cached_memory = torch.cuda.memory_reserved(i) / (1024**3)
            free_memory = total_memory - allocated_memory
            
            print(f"\nGPU {i}: {props.name}")
            print(f"  计算能力: {props.major}.{props.minor}")
            print(f"  总显存: {total_memory:.1f} GB")
            print(f"  已分配: {allocated_memory:.2f} GB")
            print(f"  已缓存: {cached_memory:.2f} GB")
            print(f"  可用显存: {free_memory:.1f} GB")
            
            # GPU评估
            if total_memory >= 80:
                gpu_status = "✅ 80GB GPU - 优秀配置"
            elif total_memory >= 40:
                gpu_status = "✅ 40GB GPU - 推荐配置"
            elif total_memory >= 24:
                gpu_status = "⚠️ 24GB GPU - 可用但受限"
            elif total_memory >= 12:
                gpu_status = "⚠️ 12GB GPU - 仅支持小模型"
            else:
                gpu_status = "❌ <12GB GPU - 不建议使用"
            
            print(f"  状态: {gpu_status}")
    else:
        print("❌ CUDA不可用，将使用CPU模式")
        print("⚠️ 警告: CPU模式下性能将显著降低")
        
except ImportError:
    print("❌ PyTorch未安装，无法检测GPU")

## 3. 依赖库检测和安装

In [ ]:
print("\n📦 关键依赖库检测:")
print("-" * 40)

# 核心依赖库列表
core_dependencies = {
    'torch': '>=1.12.0',
    'transformers': '>=4.20.0',
    'diffusers': '>=0.20.0',
    'numpy': '>=1.21.0',
    'scipy': '>=1.8.0',
    'librosa': '>=0.9.0',
    'opencv-python': '>=4.5.0',
    'pillow': '>=8.0.0',
    'matplotlib': '>=3.5.0',
    'seaborn': '>=0.11.0',
    'plotly': '>=5.0.0',
    'ipywidgets': '>=7.6.0',
    'tqdm': '>=4.60.0',
    'pyyaml': '>=5.4.0',
    'psutil': '>=5.8.0'
}

missing_packages = []
installed_packages = {}

for package, min_version in core_dependencies.items():
    try:
        if package == 'opencv-python':
            import cv2
            version = cv2.__version__
            package_name = 'cv2'
        else:
            module = __import__(package)
            version = getattr(module, '__version__', 'unknown')
            package_name = package
        
        installed_packages[package] = version
        print(f"✅ {package_name}: {version}")
        
    except ImportError:
        missing_packages.append(package)
        print(f"❌ {package}: 未安装")

if missing_packages:
    print(f"\n⚠️ 缺失依赖库: {', '.join(missing_packages)}")
    print("请运行以下命令安装:")
    print(f"pip install {' '.join(missing_packages)}")
else:
    print("\n✅ 所有核心依赖库已安装")

In [ ]:
# 可选依赖库检测 (用于特定功能)
print("\n📦 可选依赖库检测:")
print("-" * 40)

optional_dependencies = {
    'audiocraft': 'MusicGen音乐生成',
    'accelerate': 'Hugging Face模型加速',
    'xformers': '内存优化 (推荐)',
    'pynvml': 'GPU监控',
    'wandb': '实验跟踪 (可选)',
    'tensorboard': 'TensorBoard日志',
    'jupyter_contrib_nbextensions': 'Jupyter扩展',
    'ipywidgets': 'Jupyter交互组件'
}

for package, description in optional_dependencies.items():
    try:
        module = __import__(package)
        version = getattr(module, '__version__', 'unknown')
        print(f"✅ {package}: {version} - {description}")
    except ImportError:
        print(f"⚪ {package}: 未安装 - {description}")

## 4. 系统配置初始化

In [ ]:
print("\n⚙️ 系统配置初始化:")
print("-" * 40)

# 创建输出目录结构
output_dirs = [
    'outputs',
    'outputs/audio',
    'outputs/video', 
    'outputs/logs',
    'outputs/models',
    'outputs/reports',
    'outputs/cache'
]

for dir_name in output_dirs:
    dir_path = PROJECT_ROOT / dir_name
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✅ 创建目录: {dir_path}")

# 设置缓存目录
cache_dir = PROJECT_ROOT / 'outputs' / 'cache'
os.environ['TRANSFORMERS_CACHE'] = str(cache_dir / 'transformers')
os.environ['HF_HOME'] = str(cache_dir / 'huggingface')
print(f"✅ 设置缓存目录: {cache_dir}")

# 创建系统配置
system_config = {
    'project_root': str(PROJECT_ROOT),
    'python_version': sys.version,
    'platform': platform.platform(),
    'initialization_time': datetime.now().isoformat(),
    'gpu_available': torch.cuda.is_available() if 'torch' in locals() else False,
    'gpu_count': torch.cuda.device_count() if 'torch' in locals() and torch.cuda.is_available() else 0,
    'total_memory_gb': memory_gb,
    'available_memory_gb': memory_available_gb,
    'installed_packages': installed_packages,
    'output_directories': [str(PROJECT_ROOT / d) for d in output_dirs]
}

# 保存配置文件
config_file = PROJECT_ROOT / 'outputs' / 'system_config.json'
with open(config_file, 'w', encoding='utf-8') as f:
    json.dump(system_config, f, indent=2, ensure_ascii=False)

print(f"✅ 系统配置已保存: {config_file}")

## 5. 基础功能测试

In [ ]:
print("\n🧪 基础功能测试:")
print("-" * 40)

# 测试理论模块导入
try:
    from research.theory.iso_principle import ISOPrinciple, EmotionState
    from research.theory.valence_arousal import ValenceArousalModel
    from research.theory.sleep_physiology import SleepPhysiologyModel
    from research.theory.music_psychology import MusicPsychologyModel
    print("✅ 理论模块导入成功")
    
    # 快速功能测试
    emotion_state = EmotionState(valence=-0.2, arousal=0.6, confidence=0.8)
    iso_planner = ISOPrinciple()
    va_model = ValenceArousalModel()
    
    print(f"✅ 情绪状态创建: V={emotion_state.valence}, A={emotion_state.arousal}")
    print("✅ 理论模型初始化成功")
    
except Exception as e:
    print(f"❌ 理论模块测试失败: {e}")

In [ ]:
# 测试模型适配器导入
try:
    from src.models.base import BaseModelAdapter, ModelConfig, HardwareConfig
    from src.models.registry import ModelRegistry
    from src.models.factory import ModelFactory
    print("✅ 模型适配器导入成功")
    
    # 创建硬件配置
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        if gpu_memory >= 75:
            hardware_profile = "gpu_80gb"
        elif gpu_memory >= 35:
            hardware_profile = "gpu_40gb"
        else:
            hardware_profile = "gpu_low"
    else:
        hardware_profile = "cpu_only"
    
    print(f"✅ 硬件配置检测: {hardware_profile}")
    
except Exception as e:
    print(f"❌ 模型适配器测试失败: {e}")

In [ ]:
# 测试疗愈系统导入
try:
    from src.therapy.core import TherapyOrchestrator, TherapySession, EmotionTrajectoryPlanner
    from src.therapy.stages import ISOStageManager
    from src.therapy.prescriptions import PrescriptionEngine
    print("✅ 疗愈系统导入成功")
    
    # 创建处方引擎测试
    prescription_engine = PrescriptionEngine()
    print("✅ 处方引擎初始化成功")
    
except Exception as e:
    print(f"❌ 疗愈系统测试失败: {e}")

## 6. 模型缓存预热 (可选)

In [ ]:
print("\n🔥 模型缓存预热:")
print("-" * 40)
print("⚠️ 此步骤可选，首次运行可能需要较长时间下载模型")
print("建议在网络条件良好时运行")

# 预热小模型 (较快)
PRELOAD_MODELS = input("\n是否预加载小模型进行测试? (y/n): ").lower().strip() == 'y'

if PRELOAD_MODELS:
    try:
        print("\n开始预加载模型...")
        
        # 预加载文本情绪识别模型 (较小)
        print("📥 预加载文本情绪识别模型...")
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        
        model_id = "cardiffnlp/twitter-roberta-base-emotion"
        tokenizer = AutoTokenizer.from_pretrained(
            model_id, 
            cache_dir=cache_dir / 'transformers'
        )
        print(f"✅ Tokenizer加载成功: {model_id}")
        
        # 如果GPU可用且内存充足，预加载模型
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            if gpu_memory >= 8:  # 至少8GB才加载模型
                model = AutoModelForSequenceClassification.from_pretrained(
                    model_id,
                    cache_dir=cache_dir / 'transformers'
                )
                print(f"✅ 模型加载成功: {model_id}")
                
                # 释放内存
                del model
                torch.cuda.empty_cache()
                print("✅ GPU内存已清理")
            else:
                print("⚠️ GPU内存不足，跳过模型预加载")
        
        del tokenizer
        gc.collect()
        print("✅ 模型缓存预热完成")
        
    except Exception as e:
        print(f"❌ 模型预热失败: {e}")
        print("这可能是网络问题，可以稍后再试")
else:
    print("⏭️ 跳过模型预热")

## 7. 系统状态总结

In [ ]:
print("\n" + "=" * 60)
print("《心境流转》系统初始化总结")
print("=" * 60)

# 收集状态信息
status_report = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'environment': {
        'python_version': sys.version.split()[0],
        'platform': platform.system(),
        'working_directory': str(PROJECT_ROOT)
    },
    'resources': {
        'cpu_cores': cpu_count,
        'total_memory_gb': round(memory_gb, 1),
        'available_memory_gb': round(memory_available_gb, 1),
        'disk_free_gb': round(disk_free_gb, 1)
    },
    'gpu': {
        'cuda_available': torch.cuda.is_available() if 'torch' in locals() else False,
        'gpu_count': torch.cuda.device_count() if 'torch' in locals() and torch.cuda.is_available() else 0
    },
    'dependencies': {
        'core_installed': len(missing_packages) == 0,
        'missing_packages': missing_packages
    },
    'modules': {
        'theory_modules': 'iso_planner' in locals(),
        'model_adapters': 'ModelFactory' in locals(),
        'therapy_system': 'TherapyOrchestrator' in locals()
    }
}

# 添加GPU详细信息
if status_report['gpu']['cuda_available']:
    gpu_info = []
    for i in range(status_report['gpu']['gpu_count']):
        props = torch.cuda.get_device_properties(i)
        gpu_info.append({
            'name': props.name,
            'memory_gb': round(props.total_memory / (1024**3), 1),
            'compute_capability': f"{props.major}.{props.minor}"
        })
    status_report['gpu']['devices'] = gpu_info

# 显示状态报告
print(f"初始化时间: {status_report['timestamp']}")
print(f"Python版本: {status_report['environment']['python_version']}")
print(f"系统平台: {status_report['environment']['platform']}")
print(f"\n💾 系统资源:")
print(f"  CPU核心: {status_report['resources']['cpu_cores']}")
print(f"  总内存: {status_report['resources']['total_memory_gb']} GB")
print(f"  可用内存: {status_report['resources']['available_memory_gb']} GB")
print(f"  磁盘可用: {status_report['resources']['disk_free_gb']} GB")

print(f"\n🎮 GPU状态:")
if status_report['gpu']['cuda_available']:
    print(f"  CUDA可用: ✅")
    print(f"  GPU数量: {status_report['gpu']['gpu_count']}")
    for i, gpu in enumerate(status_report['gpu']['devices']):
        print(f"  GPU {i}: {gpu['name']} ({gpu['memory_gb']} GB)")
else:
    print(f"  CUDA可用: ❌")

print(f"\n📦 依赖状态:")
if status_report['dependencies']['core_installed']:
    print("  核心依赖: ✅ 全部已安装")
else:
    print(f"  核心依赖: ❌ 缺失 {len(status_report['dependencies']['missing_packages'])} 个")

print(f"\n🧩 模块状态:")
modules = status_report['modules']
print(f"  理论模块: {'✅' if modules['theory_modules'] else '❌'}")
print(f"  模型适配器: {'✅' if modules['model_adapters'] else '❌'}")
print(f"  疗愈系统: {'✅' if modules['therapy_system'] else '❌'}")

# 给出总体评估
all_good = (
    status_report['dependencies']['core_installed'] and
    modules['theory_modules'] and
    modules['model_adapters'] and
    modules['therapy_system']
)

print(f"\n🎯 系统状态:")
if all_good:
    print("✅ 系统初始化成功！可以继续运行其他notebook")
    recommendation = "建议按顺序运行：02_theory_models_demo → 03_model_adapters_test → 04_therapy_session_demo"
else:
    print("⚠️ 系统初始化存在问题，请检查上述错误信息")
    recommendation = "请先解决依赖库和模块导入问题，然后重新运行此notebook"

print(f"\n💡 建议: {recommendation}")

# 保存状态报告
status_file = PROJECT_ROOT / 'outputs' / 'initialization_status.json'
with open(status_file, 'w', encoding='utf-8') as f:
    json.dump(status_report, f, indent=2, ensure_ascii=False)

print(f"\n📄 状态报告已保存: {status_file}")
print("\n" + "=" * 60)

## 🎉 初始化完成！

如果上述所有检查都通过，您现在可以：

1. **继续运行理论演示**: `02_theory_models_demo.ipynb`
2. **测试模型适配器**: `03_model_adapters_test.ipynb`  
3. **体验完整疗愈流程**: `04_therapy_session_demo.ipynb`

### 📝 重要提示
- 所有输出文件将保存在 `outputs/` 目录
- 模型缓存位于 `outputs/cache/` 目录
- 如遇问题，请查看 `10_troubleshooting_guide.ipynb`

### 🆘 获取帮助
如果初始化过程中遇到问题，请：
1. 检查JupyterHub环境的GPU和内存资源
2. 确认网络连接正常 (模型下载需要)
3. 查看错误日志定位问题
4. 参考故障排除指南